In [99]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import LabelEncoder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import torch.nn.utils.rnn as rnn_utils

In [100]:
df = pd.read_csv("netflix_reviews.csv")  # 파일 불러오기

def preprocess_text(text):
    if isinstance(text, float):
        return ""
    text = text.lower()  # 대문자를 소문자로
    text = re.sub(r'[^\w\s]', '', text)  # 구두점 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    text = text.strip()  # 띄어쓰기 제외하고 빈 칸 제거
    return text

df['content'] = df['content'].apply(preprocess_text)  # 텍스트 전처리

#df.head()

In [101]:
X = df['content'].tolist() # 리뷰 리스트 .tolist()
y = df['score'].tolist() # 점수 리스트
#print(df['content'].head(4))
#print(df['score'].dtypes)

#어휘 정수화
def text_pipeline(text):
    return [vocab[token] for token in tokenizer(text)]

def label_pipeline(label):
    return np.int64(label)

X_tr, X_te, y_tr, y_te = tts(X,y)

In [102]:
# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    def __init__(self, X, y, text_pipeline, label_pipeline):
        self.X = X
        self.y = y
        self.text_pipeline = text_pipeline
        self.label_pipeline = label_pipeline

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        review = self.text_pipeline(self.X[idx])
        rating = self.label_pipeline(self.y[idx])
        return torch.tensor(review), torch.tensor(rating)

In [112]:
# 어휘 구축 과정
# 토크나이저 정의 (기본 영어 토크나이저)
tokenizer = get_tokenizer('basic_english')

# 어휘 사전 생성 함수
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)


# 데이터셋 정의
train_dataset = ReviewDataset(X_tr, y_tr, text_pipeline, label_pipeline)
test_dataset = ReviewDataset(X_te, y_te, text_pipeline, label_pipeline)

# 어휘 사전 생성
vocab = build_vocab_from_iterator(yield_tokens(X))

# 패딩 처리 함수 정의
def collate_batch(batch):
    reviews, labels = zip(*batch)
    reviews_padded = rnn_utils.pad_sequence(reviews, batch_first=True)
    labels_tensor = torch.tensor(labels, dtype=torch.int64)
    return reviews_padded, labels_tensor

# 데이터 로더 정의
BATCH_SIZE = 64

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

117134lines [00:02, 43489.86lines/s]


In [113]:
# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded.unsqueeze(0))
        return self.fc(hidden[-1])

# 하이퍼파라미터 정의
VOCAB_SIZE = len(vocab)
EMBED_DIM = 64
HIDDEN_DIM = 128
OUTPUT_DIM = len(set(ratings))  # 예측할 점수 개수

# 모델 초기화
model = LSTMModel(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, OUTPUT_DIM)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [114]:
# 모델 학습 함수
def train_model(model, train_dataloader, criterion, optimizer, num_epochs=5):
    model.train()  # 모델을 학습 모드로 설정
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_dataloader:
            reviews, labels = batch
            optimizer.zero_grad()  # 기울기 초기화
            output = model(reviews)  # 순전파
            loss = criterion(output, labels)  # 손실 계산
            loss.backward()  # 역전파
            optimizer.step()  # 매개변수 업데이트
            total_loss += loss.item()  # 손실 누적
        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

# 모델 평가 함수
def evaluate_model(model, test_dataloader):
    model.eval()  # 모델을 평가 모드로 설정
    total_correct = 0
    total_samples = 0
    with torch.no_grad():  # 기울기 계산 비활성화
        for batch in test_dataloader:
            reviews, labels = batch
            output = model(reviews)
            predictions = torch.argmax(output, dim=1)  # 예측 클래스 얻기
            total_correct += (predictions == labels).sum().item()  # 맞춘 예측 수 카운트
            total_samples += labels.size(0)  # 전체 샘플 수 카운트
    accuracy = total_correct / total_samples
    print(f"Test Accuracy: {accuracy:.4f}")

# 모델 학습
num_epochs = 5  # 학습할 에폭 수 조정 가능
train_model(model, train_dataloader, criterion, optimizer, num_epochs)

# 테스트 세트에서 모델 평가
evaluate_model(model, test_dataloader)


ValueError: Expected input batch_size (1) to match target batch_size (64).